# Physical Units

Braincore includes a system for physical units. The base units are defined by their standard SI unit names:
`amp`/`ampere`, `kilogram`/`kilogramme`, `second`, `metre`/`meter`, `kilogram`, `mole`/`mol`, `kelvin`, and `candela`. In addition to these base units, braincore defines a set of derived units: `coulomb`, `farad`, `gram`/`gramme`, `hertz`, `joule`, `liter`/
`litre`, `molar`, `pascal`, `ohm`,  `siemens`, `volt`, `watt`,
together with prefixed versions (e.g. `msiemens = 0.001*siemens`) using the
prefixes `p, n, u, m, k, M, G, T` (two exceptions to this rule: `kilogram`
is not defined with any additional prefixes, and `metre` and `meter` are
additionaly defined with the "centi" prefix, i.e. `cmetre`/`cmeter`).
For convenience, a couple of additional useful standard abbreviations such as
`cm` (instead of `cmetre`/`cmeter`), `nS` (instead of `nsiemens`),
`ms` (instead of `msecond`), `Hz` (instead of `hertz`), `mM`
(instead of `mmolar`) are included. To avoid clashes with common variable
names, no one-letter abbreviations are provided (e.g. you can use `mV` or
`nS`, but *not* `V` or `S`).

## Importing units
Braincore generates standard names for units, combining the unit name (e.g. “siemens”) with a prefixes (e.g. “m”), and also generates squared and cubed versions by appending a number. For example, the units “msiemens”, “siemens2”, “usiemens3” are all predefined. You can import these units from the package `briancore.units` – accordingly, an `from braincore.units import *` will result in everything being imported.

We recommend importing only the units you need, to have a cleaner namespace. For example, `import braincore.units as U` and then using `U.msiemens` instead of `msiemens`.

In [1]:
import brainunit as U

c:\Users\13107\.conda\envs\brainpy-dev\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Using units
You can generate a physical quantity by multiplying a scalar or ndarray with its physical unit:

In [2]:
tau = 20 * U.ms
tau

20. * msecond

In [3]:
rates = [10, 20, 30] * U.Hz
rates

ArrayImpl([10., 20., 30.], dtype=float32) * hertz

In [4]:
rates = [[10, 20, 30], [20, 30, 40]] * U.Hz
rates

ArrayImpl([[10., 20., 30.],
           [20., 30., 40.]], dtype=float32) * hertz

Braincore will check the consistency of operations on units and raise an error for dimensionality mismatches:

In [5]:
try:
    tau += 1  # ms? second?
except Exception as e:
    print(e)

Cannot calculate ... += 1, units do not match (units are s and 1).


In [6]:
try:
    3 * U.kgram + 3 * U.amp 
except Exception as e:
    print(e)

Cannot calculate 3.0 + 3.0, units do not match (units are kg and A).


## Basics
Numpy functions have been overwritten to correctly work with units.

The important attributes of a `Quantity` object are:
- `value`: the numerical value of the quantity
- `unit`: the unit of the quantity
- `ndim`: the number of dimensions of quantity's value
- `shape`: the shape of the quantity's value
- `size`: the size of the quantity's value
- `dtype`: the dtype of the quantity's value

### An example

In [7]:
rates

ArrayImpl([[10., 20., 30.],
           [20., 30., 40.]], dtype=float32) * hertz

In [8]:
rates.value

Array([[10., 20., 30.],
       [20., 30., 40.]], dtype=float32)

In [9]:
rates.unit

second ** -1

In [10]:
rates.ndim, rates.shape, rates.size, rates.dtype

(2, (2, 3), 6, dtype('float32'))

### Quantity Creation
Creating a Quantity object can be accomplished in several ways, categorized based on the type of input used. Here, we present the methods grouped by their input types and characteristics for better clarity.

In [11]:
import jax.numpy as jnp
import braincore as bc
bc.environ.set(precision=64) # we recommend using 64-bit precision for better numerical stability


#### Scalar and Array Multiplication
- Multiplying a Scalar with a Unit


In [12]:
5 * U.ms

5. * msecond

- Multiplying a Jax nunmpy value type with a Unit:

In [13]:
jnp.float64(5) * U.ms

5. * msecond

- Multiplying a Jax numpy array with a Unit:

In [14]:
jnp.array([1, 2, 3]) * U.ms

ArrayImpl([1., 2., 3.]) * msecond

- Multiplying a List with a Unit:

In [15]:
[1, 2, 3] * U.ms

ArrayImpl([1., 2., 3.]) * msecond

#### Direct Quantity Creation


In [16]:
from braincore.units import Quantity

- Creating a Quantity Directly with a Value

In [17]:
Quantity(5)

Quantity(5.)

- Creating a Quantity Directly with a Value and Unit

In [18]:
Quantity(5, unit=U.ms.unit)

5. * second

- Creating a Quantity with a Jax numpy Array of Values and a Unit

In [19]:
Quantity(jnp.array([1, 2, 3]), unit=U.ms.unit)

ArrayImpl([1., 2., 3.]) * second

- Creating a Quantity with a List of Values and a Unit

In [20]:
Quantity([1, 2, 3], unit=U.ms.unit)

ArrayImpl([1., 2., 3.]) * second

- Creating a Quantity with a List of Quantities

In [21]:
Quantity([500 * U.ms, 1 * U.second])

ArrayImpl([0.5, 1. ]) * second

- Using the with_units Method

In [22]:
Quantity.with_units(jnp.array([0.5, 1]), second=1)

ArrayImpl([0.5, 1. ]) * second

#### Unitless Quantity
Quantities can be unitless, which means they have no units. If there is no unit provided, the quantity is assumed to be unitless. The following are examples of creating unitless quantities:

In [23]:
Quantity([1, 2, 3])

Quantity(ArrayImpl([1., 2., 3.]))

In [24]:
Quantity(jnp.array([1, 2, 3]))

Quantity(ArrayImpl([1., 2., 3.]))

In [25]:
Quantity([])

Quantity(ArrayImpl([], dtype=float64))

#### Illegal Quantity Creation
The following are examples of illegal quantity creation:

In [26]:
try:
    Quantity([500 * U.ms, 1])
except Exception as e:
    print(e)

All elements must have the same unit


In [27]:
try:
    Quantity(["some", "nonsense"])
except Exception as e:
    print(e)

Value 'some' with dtype <U4 is not a valid JAX array type. Only arrays of numeric types are supported by JAX.


In [28]:
try:
    Quantity([500 * U.ms, 1 * U.volt])
except Exception as e:
    print(e)

All elements must have the same unit


### Basic Operations
Like Numpy and Jax numpy, arithmetic operators on arrays apply elementwise. A new array is created and filled with the result.

In [29]:
a = [20, 30, 40, 50] * U.mV
b = jnp.arange(4) * U.mV
b

ArrayImpl([0., 1., 2., 3.]) * mvolt

In [30]:
c = a - b
c

ArrayImpl([20., 29., 38., 47.]) * mvolt

In [31]:
b = b ** 2
b

ArrayImpl([0., 1., 4., 9.]) * mvolt2

In [32]:
a < 35 * U.mV

Array([ True,  True, False, False], dtype=bool)

The product operator * operates elementwise in Quantity and the matrix product can be performed using the @ operator or the dot function or method:

In [33]:
A = jnp.array([[1, 2], [3, 4]]) * U.mV
B = jnp.array([[2, 3], [4, 5]]) * U.mV

A * B

ArrayImpl([[ 2.,  6.],
           [12., 20.]]) * mvolt2

In [34]:
A @ B

ArrayImpl([[10., 13.],
           [22., 29.]]) * mvolt2

In [35]:
A.dot(B)

ArrayImpl([[10., 13.],
           [22., 29.]]) * mvolt2

When operating with arrays of different types, the type of the resulting array corresponds to the more general or precise one

In [36]:
a = jnp.ones(3, dtype=jnp.int32) * U.mV
b = jnp.linspace(0, jnp.pi, 3) * U.mV

b.dtype.name

'float64'

In [37]:
c = a + b
c.dtype.name

'float64'

Many unary operations, such as computing the sum of all the elements in the array, are implemented as methods of the Quantity class.

In [38]:
a = bc.random.random((2, 3)) * U.mV
a

ArrayImpl([[225.53381666, 346.90286176,  31.43644236],
           [258.49739247, 240.52782238, 584.5029428 ]]) * uvolt

In [39]:
a.sum()

1.68740128 * mvolt

In [40]:
a.min()

31.43644236 * uvolt

In [41]:
a.max()

0.58450294 * mvolt

By default, these operations apply to the array as though it were a list of numbers, regardless of its shape. However, by specifying the axis parameter you can apply an operation along the specified axis of a Quantity:

In [42]:
b = jnp.arange(12).reshape(3, 4) * U.mV
b

ArrayImpl([[ 0.,  1.,  2.,  3.],
           [ 4.,  5.,  6.,  7.],
           [ 8.,  9., 10., 11.]]) * mvolt

In [43]:
b.sum(axis=0) # sum of each column

ArrayImpl([12., 15., 18., 21.]) * mvolt

In [44]:
b.min(axis=1) # min of each row

ArrayImpl([0., 4., 8.]) * mvolt

In [45]:
b.cumsum(axis=1) # cumulative sum along each row

ArrayImpl([[ 0.,  1.,  3.,  6.],
           [ 4.,  9., 15., 22.],
           [ 8., 17., 27., 38.]]) * mvolt

### Universal Functions
Braincore provides familiar mathematical functions such as sin, cos, and exp. In braincore, these are called “universal functions” (ufunc). Within braincore, these functions operate elementwise on a Quantity, producing a Quantity as output.
Attention: For some ufuncs, they only work with unitless quantities. If you want to know the category of a ufunc, you can check the `braincore.math.ufuncs.py`.

In [46]:
B = Quantity(jnp.arange(3))
B

Quantity(ArrayImpl([0., 1., 2.]))

In [47]:
bc.math.exp(B)

Array([1.        , 2.71828183, 7.3890561 ], dtype=float64)

In [48]:
bc.math.sqrt(B)

Array([0.        , 1.        , 1.41421356], dtype=float64)

In [49]:
C = Quantity(jnp.arange(3))
bc.math.add(B, C)

Quantity(ArrayImpl([0., 2., 4.]))

### Indexing, Slicing, and Iterating
One-dimensional Quantity can be indexed, sliced and iterated over, much like lists and other Python sequences.

In [50]:
a = jnp.arange(10) ** 3 * U.mV
a

ArrayImpl([  0.,   1.,   8.,  27.,  64., 125., 216., 343., 512., 729.]) * mvolt

In [51]:
a[2]

8. * mvolt

In [52]:
a[2:5]

ArrayImpl([ 8., 27., 64.]) * mvolt

Only same dimension Quantity can be set to a slice of a Quantity.

In [53]:
# equivalent to a[0:6:2] = 1000;
# from start to position 6, exclusive, set every 2nd element to 1000
a[:6:2] = 1000 * U.mV
a

ArrayImpl([1000.,    1., 1000.,   27., 1000.,  125.,  216.,  343.,  512.,
            729.]) * mvolt

In [54]:
a[::-1] # reversed a

ArrayImpl([ 729.,  512.,  343.,  216.,  125., 1000.,   27., 1000.,    1.,
           1000.]) * mvolt

In [55]:
for i in a:
    print(i**(1 / 3.))

1. m^0.6666666666666666 kg^0.3333333333333333 s^-1.0 A^-0.3333333333333333
0.1 m^0.6666666666666666 kg^0.3333333333333333 s^-1.0 A^-0.3333333333333333
1. m^0.6666666666666666 kg^0.3333333333333333 s^-1.0 A^-0.3333333333333333
0.3 m^0.6666666666666666 kg^0.3333333333333333 s^-1.0 A^-0.3333333333333333
1. m^0.6666666666666666 kg^0.3333333333333333 s^-1.0 A^-0.3333333333333333
0.5 m^0.6666666666666666 kg^0.3333333333333333 s^-1.0 A^-0.3333333333333333
0.6 m^0.6666666666666666 kg^0.3333333333333333 s^-1.0 A^-0.3333333333333333
0.7 m^0.6666666666666666 kg^0.3333333333333333 s^-1.0 A^-0.3333333333333333
0.8 m^0.6666666666666666 kg^0.3333333333333333 s^-1.0 A^-0.3333333333333333
0.9 m^0.6666666666666666 kg^0.3333333333333333 s^-1.0 A^-0.3333333333333333


Multidimensional Quantity can have one index per axis. These indices are given in a tuple separated by commas:

In [56]:
def f(x, y):
    return 10 * x + y
b = jnp.fromfunction(f, (5, 4), dtype=jnp.int32) * U.mV
b

ArrayImpl([[ 0.,  1.,  2.,  3.],
           [10., 11., 12., 13.],
           [20., 21., 22., 23.],
           [30., 31., 32., 33.],
           [40., 41., 42., 43.]]) * mvolt

In [57]:
b[2, 3]

23. * mvolt

In [58]:
b[0:5, 1]  # each row in the second column of b

ArrayImpl([ 1., 11., 21., 31., 41.]) * mvolt

In [59]:
b[:, 1]  # equivalent to the previous example

ArrayImpl([ 1., 11., 21., 31., 41.]) * mvolt

In [60]:
b[1:3, :]  # each column in the second and third row of b

ArrayImpl([[10., 11., 12., 13.],
           [20., 21., 22., 23.]]) * mvolt

When fewer indices are provided than the number of axes, the missing indices are considered complete slices:

In [61]:
b[-1]

ArrayImpl([40., 41., 42., 43.]) * mvolt

The expression within brackets in b[i] is treated as an i followed by as many instances of : as needed to represent the remaining axes. NumPy also allows you to write this using dots as b[i, ...].

The dots (...) represent as many colons as needed to produce a complete indexing tuple. For example, if x is a Quantity with 5 axes, then
- x[1, 2, ...] is equivalent to x[1, 2, :, :, :],
- x[..., 3] to x[:, :, :, :, 3] and
- x[4, ..., 5, :] to x[4, :, :, 5, :].

In [62]:
c = jnp.array([[[0, 1, 2], [10, 12, 13]], [[100, 101, 102], [110, 112, 113]]]) * U.mV # a 3D array (two stacked 2D arrays)
c.shape

(2, 2, 3)

In [63]:
c[1, ...] # same as c[1, :, :] or c[1]

ArrayImpl([[100., 101., 102.],
           [110., 112., 113.]]) * mvolt

In [64]:
c[..., 2] # same as c[:, :, 2]

ArrayImpl([[  2.,  13.],
           [102., 113.]]) * mvolt

Iterating over multidimensional Quantity is done with respect to the first axis:

In [65]:
for row in b:
    print(row)

[0. 1. 2. 3.] mV
[10. 11. 12. 13.] mV
[20. 21. 22. 23.] mV
[30. 31. 32. 33.] mV
[40. 41. 42. 43.] mV


## Shape Manipulation
### Changing the shape of a Quantity
A Quantity has a shape given by the number of elements along each axis:

In [66]:
a = jnp.floor(10 * bc.random.random((3, 4))) * U.mV
a

ArrayImpl([[5., 5., 3., 3.],
           [6., 2., 3., 9.],
           [0., 3., 2., 7.]]) * mvolt

In [67]:
a.shape

(3, 4)

The shape of a Quantity can be changed with various commands. Note that the following three commands all return a modified array, but do not change the original array:

In [68]:
a.ravel()  # returns the array, flattened

ArrayImpl([5., 5., 3., 3., 6., 2., 3., 9., 0., 3., 2., 7.]) * mvolt

In [69]:
a.reshape(6, 2) # returns the array with a modified shape

ArrayImpl([[5., 5.],
           [3., 3.],
           [6., 2.],
           [3., 9.],
           [0., 3.],
           [2., 7.]]) * mvolt

In [70]:
a.T  # returns the array, transposed

ArrayImpl([[5., 6., 0.],
           [5., 2., 3.],
           [3., 3., 2.],
           [3., 9., 7.]]) * mvolt

In [71]:
a.T.shape

(4, 3)

In [72]:
a.shape

(3, 4)

The order of the elements in the Quantity resulting from ravel is normally “C-style”, that is, the rightmost index “changes the fastest”, so the element after a[0, 0] is a[0, 1]. If the Quantity is reshaped to some other shape, again the Quantity is treated as “C-style”. NumPy normally creates arrays stored in this order, so ravel will usually not need to copy its argument, but if the Quantity was made by taking slices of another Quantity or created with unusual options, it may need to be copied. The functions ravel and reshape can also be instructed, using an optional argument, to use FORTRAN-style arrays, in which the leftmost index changes the fastest.

The reshape function only returns its argument with a modified shape, due to Jax's immutability. The resize method is not available in braincore.

If a dimension is given as -1 in a reshaping operation, the other dimensions are automatically calculated:

In [73]:
a.reshape(3, -1)

ArrayImpl([[5., 5., 3., 3.],
           [6., 2., 3., 9.],
           [0., 3., 2., 7.]]) * mvolt

### Stacking together different arrays
Several arrays can be stacked together along different axes:

In [75]:
a = jnp.floor(10 * bc.random.random((2, 2))) * U.mV
a

ArrayImpl([[0., 2.],
           [4., 3.]]) * mvolt

In [76]:
b = jnp.floor(10 * bc.random.random((2, 2))) * U.mV
b

ArrayImpl([[1., 8.],
           [2., 2.]]) * mvolt

In [77]:
bc.math.vstack((a, b))

ArrayImpl([[0., 2.],
           [4., 3.],
           [1., 8.],
           [2., 2.]]) * mvolt

In [78]:
bc.math.hstack((a, b))

ArrayImpl([[0., 2., 1., 8.],
           [4., 3., 2., 2.]]) * mvolt

The function column_stack stacks 1D Quantities as columns into a 2D Quantities. It is equivalent to hstack only for 2D Quantities:

In [79]:
bc.math.column_stack((a, b)) # with 2D arrays

ArrayImpl([[0., 2., 1., 8.],
           [4., 3., 2., 2.]]) * mvolt

In [80]:
a = jnp.array([4., 2.]) * U.mV
b = jnp.array([2., 8.]) * U.mV
bc.math.column_stack((a, b)) # returns a 2D array

ArrayImpl([[4., 2.],
           [2., 8.]]) * mvolt

In [81]:
bc.math.hstack((a, b)) # the result is different

ArrayImpl([4., 2., 2., 8.]) * mvolt

In [82]:
a[:, jnp.newaxis] # view `a` as a 2D column vector

ArrayImpl([[4.],
           [2.]]) * mvolt

In [83]:
bc.math.column_stack((a[:, jnp.newaxis], b[:, jnp.newaxis]))

ArrayImpl([[4., 2.],
           [2., 8.]]) * mvolt

In [84]:
bc.math.hstack((a[:, jnp.newaxis], b[:, jnp.newaxis])) # the result is the same

ArrayImpl([[4., 2.],
           [2., 8.]]) * mvolt

### Splitting one Quantity into several smaller ones
Using hsplit, you can split an array along its horizontal axis, either by specifying the number of equally shaped Quantities to return, or by specifying the columns after which the division should occur:

In [85]:
a = jnp.floor(10 * bc.random.random((2, 12))) * U.mV
a

ArrayImpl([[9., 4., 4., 6., 3., 9., 6., 9., 7., 0., 0., 8.],
           [0., 2., 4., 4., 9., 0., 5., 0., 3., 3., 8., 8.]]) * mvolt

In [86]:
bc.math.hsplit(a, 3) # Split a into 3

[ArrayImpl([[9., 4., 4., 6.],
            [0., 2., 4., 4.]]) * mvolt,
 ArrayImpl([[3., 9., 6., 9.],
            [9., 0., 5., 0.]]) * mvolt,
 ArrayImpl([[7., 0., 0., 8.],
            [3., 3., 8., 8.]]) * mvolt]

In [87]:
bc.math.hsplit(a, (3, 4)) # Split `a` after the third and the fourth column

[ArrayImpl([[9., 4., 4.],
            [0., 2., 4.]]) * mvolt,
 ArrayImpl([[6.],
            [4.]]) * mvolt,
 ArrayImpl([[3., 9., 6., 9., 7., 0., 0., 8.],
            [9., 0., 5., 0., 3., 3., 8., 8.]]) * mvolt]

vsplit splits along the vertical axis, and array_split allows one to specify along which axis to split.

## Advanced Guide

### Displaying Quantity
Braincore provide `in_unit` method to display the Quantity in a specific unit. The following are examples of displaying Quantity in a specific unit:

In [90]:
from braincore.units import in_unit

q = 3. * U.volt
q

3. * volt

In [91]:
in_unit(q, U.mvolt)

'3000. mV'

### Unary Operations
The unary operations positive (+) ,negative (-), absolute (abs), and inversion (~) are supported:

In [92]:
q = 5. * U.mV
q

5. * mvolt

In [96]:
q = -q
q

-5. * mvolt

In [97]:
q = +q
q

-5. * mvolt

In [98]:
q = abs(q)
q

5. * mvolt

In [102]:
q1 = Quantity(0b101)
q2 = Quantity(-0b110)
q1, q2

(Quantity(5.), Quantity(-6.))

In [103]:
~q1, ~q2

(Quantity(-6.), Quantity(5.))

### Comparison
The comparison operators (<, <=, ==, !=, >, >=) are supported:

In [112]:
q1 = jnp.arange(10, 20, 2) * U.mV
q2 = jnp.arange(8, 27, 4) * U.mV
q1, q2

(ArrayImpl([10., 12., 14., 16., 18.]) * mvolt,
 ArrayImpl([ 8., 12., 16., 20., 24.]) * mvolt)

In [116]:
q1 == q2, q1 != q2

(Array([False,  True, False, False, False], dtype=bool),
 Array([ True, False,  True,  True,  True], dtype=bool))

In [117]:
q1 < q2, q1 <= q2

(Array([False, False,  True,  True,  True], dtype=bool),
 Array([False,  True,  True,  True,  True], dtype=bool))

In [118]:
q1 > q2, q1 >= q2

(Array([ True, False, False, False, False], dtype=bool),
 Array([ True,  True, False, False, False], dtype=bool))

### Binary Operations
The binary operations add (+), subtract (-), multiply (*), divide (/), floor divide (//), remainder (%), divmod (divmod), power (**), matmul (@), shift (<<, >>), round(round) are supported:

In [134]:
q1 = jnp.array([1, 2, 3]) * U.mV
q2 = jnp.array([2, 3, 4]) * U.mV
q1, q2

(ArrayImpl([1., 2., 3.]) * mvolt, ArrayImpl([2., 3., 4.]) * mvolt)

In [135]:
q1 + q2, q1 - q2

(ArrayImpl([3., 5., 7.]) * mvolt, ArrayImpl([-1., -1., -1.]) * mvolt)

In [136]:
q1 * q2

ArrayImpl([ 2.,  6., 12.]) * mvolt2

In [137]:
q1 / q2, q1 // q2, q1 % q2

(Array([0.5       , 0.66666667, 0.75      ], dtype=float64),
 Array([0., 0., 0.], dtype=float64),
 ArrayImpl([1., 2., 3.]) * mvolt)

In [138]:
divmod(q1, q2)

(Array([0., 0., 0.], dtype=float64), ArrayImpl([1., 2., 3.]) * mvolt)

In [139]:
q1 ** 2

ArrayImpl([1., 4., 9.]) * mvolt2

In [140]:
q1 @ q2

20. * mvolt2

In [132]:
q1 = Quantity(jnp.arange(5, dtype=jnp.int32), unit=U.mV.unit)
q1 << 2, q1 >> 2

(ArrayImpl([ 0.,  4.,  8., 12., 16.]) * volt,
 ArrayImpl([0., 0., 0., 0., 1.]) * volt)

In [151]:
q1 = 80.23456 * U.mV
q2 = 100.000056 * U.mV
q3 = -100.000056 * U.mV
print("round(80.23456, 2) : ", q1.round(5))
print("round(100.000056, 3) : ", q2.round(6))
print("round(-100.000056, 3) : ", q3.round(6))

round(80.23456, 2) :  80.23 mV
round(100.000056, 3) :  100. mV
round(-100.000056, 3) :  -100. mV


### Shape Manipulation
The shape of an array can be changed with various commands. Note that the following three commands all return a modified array, but do not change the original array:

In [152]:
q = [[1, 2], [3, 4]] * U.mV
q

ArrayImpl([[1., 2.],
           [3., 4.]]) * mvolt

In [153]:
q.flatten()

ArrayImpl([1., 2., 3., 4.]) * mvolt

In [154]:
q.swapaxes(0, 1)

ArrayImpl([[1., 3.],
           [2., 4.]]) * mvolt

In [156]:
q.take(jnp.array([0, 2]))

ArrayImpl([1., 3.]) * mvolt

In [157]:
q.transpose()

ArrayImpl([[1., 3.],
           [2., 4.]]) * mvolt

In [158]:
q.tile(2)

ArrayImpl([[1., 2., 1., 2.],
           [3., 4., 3., 4.]]) * mvolt

In [159]:
q.unsqueeze(0)

ArrayImpl([[[1., 2.],
            [3., 4.]]]) * mvolt

In [162]:
q.expand_dims(0)

ArrayImpl([[[1., 2.],
            [3., 4.]]]) * mvolt

In [163]:
expand_as_shape = (1, 2, 2)
q.expand_as(jnp.zeros(expand_as_shape).shape)

ArrayImpl([[[1., 2.],
            [3., 4.]]]) * mvolt

In [173]:
q_put = [[1, 2], [3, 4]] * U.mV
q_put.put([[1, 0], [0, 1]], [10, 30] * U.mV)
q_put

ArrayImpl([[ 1., 30.],
           [10.,  4.]]) * mvolt

In [174]:
q_squeeze = [[1, 2], [3, 4]] * U.mV
q_squeeze.squeeze()

ArrayImpl([[1., 2.],
           [3., 4.]]) * mvolt

In [175]:
q_spilt = [[1, 2], [3, 4]] * U.mV
q_spilt.split(2)

[ArrayImpl([[1., 2.]]) * mvolt, ArrayImpl([[3., 4.]]) * mvolt]

### Numpy Methods
All methods that make sense on quantities should work, i.e. they check for the correct units of their arguments and return quantities with units were appropriate.

These methods defined at `braincore.math`, so you can use them by importing `import braincore.math as bm` and then using `bm.method_name`.
#### Functions that remove unit
- all
- any
- nonzero
- argmax
- argmin
- argsort
- ones_like
- zeros_like

#### Functions that keep unit
- round
- std
- sum
- trace
- cumsum
- diagonal
- max
- mean
- min
- ptp
- ravel
- absolute
- rint
- negative
- positive
- conj
- conjugate
- floor
- ceil
- trunc

#### Functions that change unit
- var
- multiply
- divide
- true_divide
- floor_divide
- dot
- matmul
- sqrt
- square
- reciprocal

#### Functions that need to match unit
- add
- subtract
- maximum
- minimum
- remainder
- mod
- fmod

#### Functions that only work with unitless quantities
- sin
- sinh
- arcsinh
- cos
- cosh
- arccos
- arccosh
- tan
- tanh
- arctan
- arctanh
- log
- log10
- exp
- expm1
- log1p

#### Functions that compare quantities
- less
- less_equal
- greater
- greater_equal
- equal
- not_equal

#### Functions that work on all quantities and return boolean arrays(Logical operations)
- logical_and
- logical_or
- logical_xor
- logical_not
- isreal
- iscomplex
- isfinite
- isinf
- isnan